In [1]:
# from flask_other.app import app
from flask import Flask
from flask_socketio import SocketIO, emit,join_room,leave_room
from threading import Lock
import os,sys
# sys.path.append('/home/kai/data/wei/Chatbot1.0/')
sys.path.append('../../../../../Chatbot1.0/')
# sys.path.append('/home/kai/data/Chatbot1.0/API/v1/frontend/')
app=Flask("demo")
from flask import  render_template,request
# from app.mong_database import MongoManager
import time
import re
# mongo_db=MongoManager(server_ip='chatbotdb')

import urllib.request
from urllib.parse import quote
import pandas as pd
import numpy as np
import time
import re
import requests

from threading import Lock


##################################
url = 'http://localhost:8889/chatbotv1'


async_mode = None

app = Flask(__name__)
app.config['SECRET_KEY'] = 'secret!'
socketio = SocketIO(app, async_mode=async_mode)




thread = None
thread_lock = Lock()
clients = []


# 后台线程 产生数据，即刻推送至前端
def background_thread():
    """Example of how to send server generated events to clients."""
    count = 0
    while True:
        socketio.sleep(5)
        count += 1
        t = time.strftime('%M:%S', time.localtime()) # 获取系统时间（只取分:秒）
        cpus = psutil.cpu_percent(interval=None, percpu=True) # 获取系统cpu使用率 non-blocking
        socketio.emit('server_response',
                      {'data': [t, *cpus], 'count': count},
                      namespace='/test') # 注意：这里不需要客户端连接的上下文，默认 broadcast = True ！！！！！！！


@app.route('/')
def index():
    return render_template('index.html', async_mode=socketio.async_mode)


@socketio.on('client_send',namespace='/test')
def client_msg(msg):  
    print(msg['data']) # this the contentext sent by client
    response = 'data get'
    socketio.emit('my_response',{'data':response},room = request.sid, namespace='/test')

#sned message to a specific user
def unique_message(uid):
    socketio.emit('specific_messgae',{'data':'are you still here'}, room=uid)
    
@socketio.on('disconnect',namespace='/test')
def disconnect():
    uid = request.sid
    print(uid + " disconnect")
    clients.remove(uid)
    last = len(clients)
    socketio.emit('unique_message',{'data':'are you still here'}, room = clients[last-1],namespace = '/test')
    
@socketio.on('connect',namespace='/test')
def connect():
    print('connect')
    print(request.sid)
    uid = request.sid
    clients.append(uid)
    print(len(clients))
    
    join_room(uid)
    
    print('above is room')
    socketio.emit('my_response',{'data':'hello'},room = uid,namespace='/test') #the first sentence 




In [ ]:
print('http://10.0.24.31:8899/')
socketio.run(app,'0.0.0.0',8899)

http://10.0.24.31:8899/
connect
bf91e94859cb4bb09be938e1fd2febc5
1
above is room
connect
a09cb1d590e44cd58781b9fbe0c2e555
2
above is room
connect
f600b28eca0641bebd442f08fed6e1ff
3
above is room
bf91e94859cb4bb09be938e1fd2febc5 disconnect
a09cb1d590e44cd58781b9fbe0c2e555 disconnect
